In [ ]:
import pandas as pd
from datetime import datetime
from arcpy.sa import *
import sys,os

In [ ]:
path = os.path.realpath('.......')
files = os.listdir()
curDir = os.getcwd()
path_curDir_files = [os.path.join(curDir, file) for file in files]
num_files = range(len(files))
path_file_dic = {k:v for (k,v) in zip(files,path_curDir_files)}

In [ ]:
path_file_dic

In [ ]:
# for file in files:
#     print(os.path.join(curDir,file))

In [ ]:
path_file["MBG_units_characteristics.gdb"]

In [ ]:
# help(os)

### TO DO

- Union Municipalities by GISCO ID, Sum of inhabitants
- Elevation
- Slope
- Area
- Distance from closest sea
- Join Company points
- Length of cycleway

### Data preparation

In [ ]:
arcpy.env.workspace

In [ ]:
arcpy.Exists("vzorek_cities")

In [ ]:
[f.name for f in arcpy.ListFields("MUNIeu_info_all_ETRS_LCC")]

In [ ]:
# Creating list of IDs flom Excel for selection of municipalities
vzorek_cities = r'E:\ESF_data\vzorek_cities.xlsx'
read_vzorek_cities = pd.read_excel(vzorek_cities)
df_vzorek_cities = pd.DataFrame(read_vzorek_cities)
IDs_vzorek_cities = df_vzorek_cities["GISCO_ID"].tolist()

In [ ]:
df_vzorek_cities.axes

In [ ]:
"CZ_554782" in IDs_vzorek_cities

In [ ]:
IDs_vzorek_cities.remove("CZ_554782")

In [ ]:
# First selection by ID from list (Excel)
arcpy.management.SelectLayerByAttribute("MUNIeu_info_all_ETRS_LCC", "NEW_SELECTION", "MUNIeu_info_all_ETRS_LCC.all__GISCO = 'CZ_554782'", None)

In [ ]:
len(IDs_vzorek_cities)

In [ ]:
#temp_lst = ["AT_10408", "AT_10420", "AT_10422"]

In [ ]:
"""
nefunguje
start_time = datetime.now()

for id_ in temp_lst[:]:
    #select = "'all__GISCO' = '{}'".format(id_)
    #select = "'all__GISCO' = 'CZ_554782'"
    #arcpy.SelectLayerByAttribute_management("MUNIeu_info_all_ETRS_LCC", "ADD_TO_SELECTION", where_clause=select)
    arcpy.management.SelectLayerByAttribute("MUNIeu_info_all_ETRS_LCC", "ADD_TO_SELECTION", "MUNIeu_info_all_ETRS_LCC.all__GISCO = 'id_'", None)

    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
"""

In [ ]:
#arcpy.management.SelectLayerByAttribute("MUNIeu_info_all_ETRS_LCC", "ADD_TO_SELECTION", "MUNIeu_info_all_ETRS_LCC.all__GISCO = 'CZ_554782'", None)

In [ ]:
# Next selection by ID from list of IDs (Excel)
print("Start:", datetime.now())
start_time = datetime.now()


fc = "MUNIeu_info_all_ETRS_LCC"
for id_ in IDs_vzorek_cities:
    field = arcpy.AddFieldDelimiters(fc, "all__GISCO")
    selection = "{field} = '{val}'".format(field=field, val=id_)
    arcpy.SelectLayerByAttribute_management(fc, "ADD_TO_SELECTION", selection)
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print("End:", datetime.now())

In [ ]:
#arcpy.CreateFileGDB_management("E:\ESF_data\ESF_data_view", "EU_units_characteristics_2.gdb")

In [ ]:
# Export selected Features
arcpy.FeatureClassToFeatureClass_conversion(fc, 
                                            "E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb", 
                                            "vzorek_cities")

# Administrative units

### Union Municipalities by GISCO ID, Sum of inhabitants

In [ ]:
arcpy.Exists("vzorek_cities")

In [ ]:
[f.name for f in arcpy.ListFields("vzorek_cities")]

In [ ]:
# Union of municipalities by IDs, suma of population (2020)
#with arcpy.EnvManager(scratchWorkspace=r"E:\ESF_data\ESF_data_view\Default.gdb", workspace=r"E:\ESF_data\ESF_data_view\Default.gdb"):
arcpy.management.Dissolve("vzorek_cities", r"E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb\vzorek_cities_dissGISCO", "all__GISCO;all__LAU18", "all__pop20 SUM", "MULTI_PART", "DISSOLVE_LINES")

### Elevation

In [ ]:
arcpy.ia.ZonalStatisticsAsTable("vzorek_cities_dissGISCO", "all__GISCO", "EU_DTM_Lamb", r"E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb\vzorek_cities_Ele", "DATA", "MIN_MAX_MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")

In [ ]:
arcpy.management.AddJoin("vzorek_cities_dissGISCO", "all__GISCO", "vzorek_cities_Ele", "all__GISCO", "KEEP_ALL", "INDEX_JOIN_FIELDS")

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion("vzorek_cities_dissGISCO", 
                                            "E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb", 
                                            "vzorek_cities_dissGISCO_Ele")

### Slope

In [ ]:
arcpy.ia.ZonalStatisticsAsTable("vzorek_cities_dissGISCO_Ele", "all__GISCO", "EU_DTM_Slope", r"E:\ESF_data\ESF_data_view\Default.gdb\vzorek_cities_Sl", "DATA", "MIN_MAX_MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")

In [ ]:
arcpy.management.AddJoin("vzorek_cities_dissGISCO_Ele", "all__GISCO", "vzorek_cities_Sl", "all__GISCO", "KEEP_ALL", "INDEX_JOIN_FIELDS")

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion("vzorek_cities_dissGISCO_Ele", 
                                            "E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb", 
                                            "vzorek_cities_dissGISCO_Ele_Sl")

### Area

In [ ]:
"""
cursor = arcpy.SearchCursor(fc1)
for row in cursor:
    print(row.getValue("all__LAU18"), end=" ")
    print(row.getValue("Area"), end=" ")
    print(row.getValue("Area_1"))
"""

In [ ]:
arcpy.management.AddField("vzorek_cities_dissGISCO_Ele_Sl", "Area_km", "DOUBLE", 10, 2, None, "Area_km", "NULLABLE", "NON_REQUIRED", '')

In [ ]:
arcpy.management.CalculateGeometryAttributes("vzorek_cities_dissGISCO_Ele_Sl", "Area_km AREA", '', "SQUARE_KILOMETERS", 'PROJCS["Europe_Lambert_Conformal_Conic",GEOGCS["GCS_European_1950",DATUM["D_European_1950",SPHEROID["International_1924",6378388.0,297.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",10.0],PARAMETER["Standard_Parallel_1",43.0],PARAMETER["Standard_Parallel_2",62.0],PARAMETER["Latitude_Of_Origin",30.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

In [ ]:
"""

fc1 = "vzorek_cities_dissGISCO_Ele_Sl"
fields = [f.name for f in arcpy.ListFields(fc1)]
with arcpy.da.UpdateCursor(fc, fields) as cur:
    for row in cur:
        row.setValue("AREA_km", row.getValue("AREA") / 1000000)
        rows.updateRow(row)

del row
del rows
"""

### Cycleway lenght

In [ ]:
arcpy.analysis.SummarizeWithin("vzorek_cities_dissGISCO_Ele_Sl", "cycleway_LCC", r"E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb\vzorek_cities_dissGISCO_Ele_Sl_Cycl", "KEEP_ALL", "Shape_Length Sum", "ADD_SHAPE_SUM", "KILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

### Distance from closest sea

In [ ]:
arcpy.analysis.Near("vzorek_cities_dissGISCO_Ele_Sl_Cycl", "CoastL", None, "LOCATION", "ANGLE", "PLANAR", "NEAR_FID NEAR_FID;NEAR_DIST NEAR_DIST_m;NEAR_X NEAR_X;NEAR_Y NEAR_Y;NEAR_ANGLE NEAR_ANGLE")

### BikeShareMap - get number of stations (SpatialJoin)

In [ ]:
arcpy.analysis.SpatialJoin("vzorek_cities_dissGISCO_Ele_Sl_Cycl", "BikeSharing_stations", r"E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb\vzorek_cities_dissGISCO_Ele_Sl_Cycl_Stat", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'all__GISCO "all__GISCO" true true false 254 Text 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,all__GISCO,0,254;all__LAU18 "all__LAU18" true true false 254 Text 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,all__LAU18,0,254;SUM_all__pop20 "SUM_all__pop20" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,SUM_all__pop20,-1,-1;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,OBJECTID,-1,-1;all__GISCO_1 "Ele_all__GISCO" true true false 254 Text 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,all__GISCO_1,0,254;ZONE_CODE "Ele_ZONE_CODE" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,ZONE_CODE,-1,-1;COUNT "Ele_COUNT" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,COUNT,-1,-1;AREA "Ele_AREA" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,AREA,-1,-1;MIN "Ele_MIN" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MIN,-1,-1;MAX "Ele_MAX" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MAX,-1,-1;MEAN "Ele_MEAN" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MEAN,-1,-1;OBJECTID_1 "OBJECTID" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,OBJECTID_1,-1,-1;all__GISCO_12 "all__GISCO" true true false 254 Text 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,all__GISCO_12,0,254;ZONE_CODE_1 "Sl_ZONE_CODE" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,ZONE_CODE_1,-1,-1;COUNT_1 "Sl_COUNT" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,COUNT_1,-1,-1;AREA_1 "Sl_AREA" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,AREA_1,-1,-1;MIN_1 "Sl_MIN" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MIN_1,-1,-1;MAX_1 "Sl_MAX" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MAX_1,-1,-1;MEAN_1 "Sl_MEAN" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,MEAN_1,-1,-1;Area_km "Area_km" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,Area_km,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,Shape_Area,-1,-1;sum_shape_length "Cycl_length_m" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,sum_shape_length,-1,-1;sum_Length_KILOMETERS "Cycl_length_km" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,sum_Length_KILOMETERS,-1,-1;Polyline_Count "Cycl_Count of Lines" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,Polyline_Count,-1,-1;NEAR_FID "NEAR_FID" true true false 4 Long 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,NEAR_FID,-1,-1;NEAR_DIST_m "NEAR_DIST_m" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,NEAR_DIST_m,-1,-1;NEAR_X "NEAR_X" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,NEAR_X,-1,-1;NEAR_Y "NEAR_Y" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,NEAR_Y,-1,-1;NEAR_ANGLE "NEAR_ANGLE" true true false 8 Double 0 0,First,#,vzorek_cities_dissGISCO_Ele_Sl_Cycl,NEAR_ANGLE,-1,-1;timestamp_ "timestamp|" true true false 80 Text 0 0,First,#,BikeSharing_stations,timestamp_,0,80;name "name" true true false 152 Text 0 0,First,#,BikeSharing_stations,name,0,152;lat "lat" true true false 80 Text 0 0,First,#,BikeSharing_stations,lat,0,80;lon "lon" true true false 80 Text 0 0,First,#,BikeSharing_stations,lon,0,80;bikes "bikes" true true false 80 Text 0 0,First,#,BikeSharing_stations,bikes,0,80;spaces "spaces" true true false 80 Text 0 0,First,#,BikeSharing_stations,spaces,0,80;installed "installed" true true false 80 Text 0 0,First,#,BikeSharing_stations,installed,0,80;locked "locked" true true false 80 Text 0 0,First,#,BikeSharing_stations,locked,0,80;temporary "temporary" true true false 80 Text 0 0,First,#,BikeSharing_stations,temporary,0,80;total_dock "total_dock" true true false 80 Text 0 0,First,#,BikeSharing_stations,total_dock,0,80', "INTERSECT", None, '')

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion("vzorek_cities_dissGISCO_Ele_Sl_Cycl_Stat", 
                                            "E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb", 
                                            "EU_units_charac")

### Company points (Join)

In [ ]:
arcpy.analysis.SpatialJoin("EU_units_charac", "Company_Points_EU", r"E:\ESF_data\ESF_data_view\data_prep_vzorek_cities.gdb\EU_units_charac_companies", "JOIN_ONE_TO_MANY", "KEEP_ALL", 'BikeShareMap_stations "BikeShareMap_stations" true true false 4 Long 0 0,First,#,EU_units_charac,BikeShareMap_stations,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,EU_units_charac,TARGET_FID,-1,-1;all__GISCO "all__GISCO" true true false 254 Text 0 0,First,#,EU_units_charac,all__GISCO,0,254;all__LAU18 "all__LAU18" true true false 254 Text 0 0,First,#,EU_units_charac,all__LAU18,0,254;SUM_all__pop20 "SUM_all__pop20" true true false 8 Double 0 0,First,#,EU_units_charac,SUM_all__pop20,-1,-1;OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,EU_units_charac,OBJECTID,-1,-1;all__GISCO_1 "Ele_all__GISCO" true true false 254 Text 0 0,First,#,EU_units_charac,all__GISCO_1,0,254;ZONE_CODE "Ele_ZONE_CODE" true true false 4 Long 0 0,First,#,EU_units_charac,ZONE_CODE,-1,-1;COUNT "Ele_COUNT" true true false 8 Double 0 0,First,#,EU_units_charac,COUNT,-1,-1;AREA "Ele_AREA" true true false 8 Double 0 0,First,#,EU_units_charac,AREA,-1,-1;MIN "Ele_MIN" true true false 4 Long 0 0,First,#,EU_units_charac,MIN,-1,-1;MAX "Ele_MAX" true true false 4 Long 0 0,First,#,EU_units_charac,MAX,-1,-1;MEAN "Ele_MEAN" true true false 8 Double 0 0,First,#,EU_units_charac,MEAN,-1,-1;OBJECTID_1 "OBJECTID" true true false 4 Long 0 0,First,#,EU_units_charac,OBJECTID_1,-1,-1;all__GISCO_12 "all__GISCO" true true false 254 Text 0 0,First,#,EU_units_charac,all__GISCO_12,0,254;ZONE_CODE_1 "Sl_ZONE_CODE" true true false 4 Long 0 0,First,#,EU_units_charac,ZONE_CODE_1,-1,-1;COUNT_1 "Sl_COUNT" true true false 8 Double 0 0,First,#,EU_units_charac,COUNT_1,-1,-1;AREA_1 "Sl_AREA" true true false 8 Double 0 0,First,#,EU_units_charac,AREA_1,-1,-1;MIN_1 "Sl_MIN" true true false 8 Double 0 0,First,#,EU_units_charac,MIN_1,-1,-1;MAX_1 "Sl_MAX" true true false 8 Double 0 0,First,#,EU_units_charac,MAX_1,-1,-1;MEAN_1 "Sl_MEAN" true true false 8 Double 0 0,First,#,EU_units_charac,MEAN_1,-1,-1;Area_km "Area_km" true true false 8 Double 0 0,First,#,EU_units_charac,Area_km,-1,-1;sum_shape_length "Cycl_length_m" true true false 8 Double 0 0,First,#,EU_units_charac,sum_shape_length,-1,-1;sum_Length_KILOMETERS "Cycl_length_km" true true false 8 Double 0 0,First,#,EU_units_charac,sum_Length_KILOMETERS,-1,-1;Polyline_Count "Cycl_Count of Lines" true true false 4 Long 0 0,First,#,EU_units_charac,Polyline_Count,-1,-1;NEAR_FID "NEAR_FID" true true false 4 Long 0 0,First,#,EU_units_charac,NEAR_FID,-1,-1;NEAR_DIST_m "NEAR_DIST_m" true true false 8 Double 0 0,First,#,EU_units_charac,NEAR_DIST_m,-1,-1;NEAR_X "NEAR_X" true true false 8 Double 0 0,First,#,EU_units_charac,NEAR_X,-1,-1;NEAR_Y "NEAR_Y" true true false 8 Double 0 0,First,#,EU_units_charac,NEAR_Y,-1,-1;NEAR_ANGLE "NEAR_ANGLE" true true false 8 Double 0 0,First,#,EU_units_charac,NEAR_ANGLE,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,EU_units_charac,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,EU_units_charac,Shape_Area,-1,-1;name_stati "name_stati" true true false 86 Text 0 0,First,#,Company_Points_EU,name_stati,0,86;num_of_sta "num_of_sta" true true false 8 Double 0 0,First,#,Company_Points_EU,num_of_sta,-1,-1;lon "lon" true true false 8 Double 0 0,First,#,Company_Points_EU,lon,-1,-1;lat "lat" true true false 8 Double 0 0,First,#,Company_Points_EU,lat,-1,-1;num_of_bik "num_of_bik" true true false 8 Double 0 0,First,#,Company_Points_EU,num_of_bik,-1,-1;launch "launch" true true false 8 Double 0 0,First,#,Company_Points_EU,launch,-1,-1;terminated "terminated" true true false 8 Double 0 0,First,#,Company_Points_EU,terminated,-1,-1;city_compa "city_compa" true true false 80 Text 0 0,First,#,Company_Points_EU,city_compa,0,80;a "a" true true false 150 Text 0 0,First,#,Company_Points_EU,a,0,150;bikes "bikes" true true false 80 Text 0 0,First,#,Company_Points_EU,bikes,0,80;stations "stations" true true false 80 Text 0 0,First,#,Company_Points_EU,stations,0,80;inc_pedele "inc_pedele" true true false 80 Text 0 0,First,#,Company_Points_EU,inc_pedele,0,80;b "b" true true false 80 Text 0 0,First,#,Company_Points_EU,b,0,80;c "c" true true false 80 Text 0 0,First,#,Company_Points_EU,c,0,80;web_compan "web_compan" true true false 139 Text 0 0,First,#,Company_Points_EU,web_compan,0,139;d "d" true true false 254 Text 0 0,First,#,Company_Points_EU,d,0,254;working "working" true true false 80 Text 0 0,First,#,Company_Points_EU,working,0,80;date_launc "date_launc" true true false 80 Text 0 0,First,#,Company_Points_EU,date_launc,0,80;date_termi "date_termi" true true false 80 Text 0 0,First,#,Company_Points_EU,date_termi,0,80;e "e" true true false 204 Text 0 0,First,#,Company_Points_EU,e,0,204;operator "operator" true true false 80 Text 0 0,First,#,Company_Points_EU,operator,0,80;equipment "equipment" true true false 80 Text 0 0,First,#,Company_Points_EU,equipment,0,80;f "f" true true false 80 Text 0 0,First,#,Company_Points_EU,f,0,80;pic "pic" true true false 251 Text 0 0,First,#,Company_Points_EU,pic,0,251;g "g" true true false 80 Text 0 0,First,#,Company_Points_EU,g,0,80', "INTERSECT", None, '')

# Minimum Bounding Geometry (MBG) areas by stations

#### Elevation for MBG

In [ ]:
arcpy.env.workspace

In [ ]:
arcpy.Exists("MBG_Pts_units")

In [ ]:
arcpy.Exists("eurolst_clim_bio01.tif")

In [ ]:
MBG_Pts_units = "MBG_Pts_units"

In [ ]:
arcpy.ia.ZonalStatisticsAsTable(MBG_Pts_units,
                                "all__GISCO",
                                "EU_DTM_Lamb",
                                #"{}\MBG_tab_Ele".format(path_file["MBG_units_characteristics.gdb"]),
                                f"{path_file_dic['MBG_units_characteristics.gdb']}\MBG_tab_Ele",
                                "DATA",
                                "MIN_MAX_MEAN",
                                "CURRENT_SLICE",
                                90,
                                "AUTO_DETECT"
                                )

In [ ]:
arcpy.ia.ZonalStatisticsAsTable(MBG_Pts_units,
                                "all__GISCO",
                                "EU_DTM_Lamb",
                                r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_tab_Ele",
                                "DATA",
                                "MIN_MAX_MEAN",
                                "CURRENT_SLICE",
                                90,
                                "AUTO_DETECT"
                                )

In [ ]:
[f.name for f in arcpy.ListFields(MBG_Pts_units)]

In [ ]:
[f.name for f in arcpy.ListFields("MBG_tab_Ele")]

In [ ]:
arcpy.management.AddJoin(MBG_Pts_units,
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "MBG_tab_Ele",
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "KEEP_ALL",
                         "INDEX_JOIN_FIELDS")

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(MBG_Pts_units, 
                                            r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb", 
                                            "MBG_units_Ele")

#### Slope for MBG

In [ ]:
arcpy.ia.ZonalStatisticsAsTable("MBG_units_Ele",
                                "all__GISCO",
                                "EU_DTM_Slope",
                                r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_tab_Sl",
                                "DATA",
                                "MIN_MAX_MEAN",
                                "CURRENT_SLICE",
                                90,
                                "AUTO_DETECT")

In [ ]:
[f.name for f in arcpy.ListFields("MBG_units_Ele")]

In [ ]:
[f.name for f in arcpy.ListFields("MBG_tab_Sl")]

In [ ]:
arcpy.management.AddJoin("MBG_units_Ele",
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "MBG_tab_Sl",
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "KEEP_ALL",
                         "INDEX_JOIN_FIELDS")

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion("MBG_units_Ele", 
                                            r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb", 
                                            "MBG_units_Ele_Sl")

In [ ]:
#arcpy.MakeFeatureLayer_management("MBG_Pts_units_Ele_lyr", "MBG_Pts_units_Ele_Sl_lyr")

#### MBG - Surface Temperature

In [ ]:
arcpy.ia.ZonalStatisticsAsTable("MBG_units_Ele_Sl",
                                "all__GISCO",
                                "eurolst_clim_bio01.tif",
                                r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_tab_Tem",
                                "DATA",
                                "MIN_MAX_MEAN",
                                "CURRENT_SLICE",
                                90,
                                "AUTO_DETECT")

In [ ]:
arcpy.management.AddJoin("MBG_units_Ele_Sl",
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "MBG_tab_Tem",
                         "MUNIeu_info_all_ETRS_LCC_all__GISCO",
                         "KEEP_ALL",
                         "INDEX_JOIN_FIELDS")

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion("MBG_units_Ele_Sl", 
                                            r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb", 
                                            "MBG_units_Ele_Sl_Tem")

In [ ]:
temp_fld_names = [f.name for f in arcpy.ListFields("MBG_units_Ele_Sl_Tem")]

In [ ]:
for idx, fld_name in enumerate(temp_fld_names):
    print(idx, fld_name)

In [ ]:
value_lst = ["MIN", "MAX", "MEAN"]
value_dic = {24:"MIN", 25:"MAX", 26:"MEAN"}

In [ ]:
for val in value_lst:
    arcpy.management.AddField("MBG_units_Ele_Sl_Tem",
                          f"Temp_{val}_C",
                          "DOUBLE",
                          10,
                          2,
                          None,
                          f"Temp_{val}_C",
                          "NULLABLE",
                          "NON_REQUIRED",
                          '')

In [ ]:
for val in range(24,27):
    #print(val)
    arcpy.management.CalculateField("MBG_units_Ele_Sl_Tem",
                                    f"Temp_{value_dic[val]}_C",
                                    f"!{temp_fld_names[val]}!/10",
                                    "PYTHON3")
    

In [ ]:
# for val2 in temp_fld_names[24:27]:
#     for val1 in value_lst:
#         arcpy.management.CalculateField("MBG_units_Ele_Sl_Tem",
#                                     f"Temp_{val1}_C",
#                                     f"!{val2}!//10",
#                                     "PYTHON3")

#### Area for MBG

In [ ]:
arcpy.management.AddField("MBG_units_Ele_Sl_Tem",
                          "Area_km",
                          "DOUBLE",
                          10,
                          2,
                          None,
                          "Area_km",
                          "NULLABLE",
                          "NON_REQUIRED",
                          '')

In [ ]:
arcpy.management.CalculateGeometryAttributes("MBG_units_Ele_Sl_Tem",
                                             "Area_km AREA",
                                             '',
                                             "SQUARE_KILOMETERS",
                                             'PROJCS["Europe_Lambert_Conformal_Conic",GEOGCS["GCS_European_1950",DATUM["D_European_1950",SPHEROID["International_1924",6378388.0,297.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",10.0],PARAMETER["Standard_Parallel_1",43.0],PARAMETER["Standard_Parallel_2",62.0],PARAMETER["Latitude_Of_Origin",30.0],UNIT["Meter",1.0]]',
                                             "SAME_AS_INPUT")

#### Cycleway lenght for MBG

In [ ]:
arcpy.analysis.SummarizeWithin("MBG_units_Ele_Sl_Tem",
                               "cycleway_LCC",
                               r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl",
                               "KEEP_ALL",
                               "Shape_Length Sum",
                               "ADD_SHAPE_SUM",
                               "KILOMETERS",
                               None,
                               "NO_MIN_MAJ",
                               "NO_PERCENT",
                               None)

#### MBG - distance from closest sea

In [ ]:
arcpy.Exists("CoastL_LCC")

In [ ]:
arcpy.analysis.Near("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl",
                    "CoastL_LCC",
                    None,
                    "LOCATION",
                    "ANGLE",
                    "PLANAR",
                    "NEAR_FID NEAR_FID;NEAR_DIST NEAR_DIST_m;NEAR_X NEAR_X;NEAR_Y NEAR_Y;NEAR_ANGLE NEAR_ANGLE")

#### MBG - BikeShareMap - get number of stations (SpatialJoin)

In [ ]:
arcpy.analysis.SpatialJoin("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl",
                           "BikeSharing_stations_LCC",
                           r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat",
                           join_operation="JOIN_ONE_TO_ONE",
                           join_type="KEEP_ALL",
                           match_option="INTERSECT"
                           )

#### MBG - count population from points' attributes (TotP)

In [ ]:
arcpy.analysis.SummarizeWithin("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat",
                               "Population_LCC",
                               r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat_TotP",
                               "KEEP_ALL",
                               "TOT_P Sum",
                               "ADD_SHAPE_SUM",
                               '',
                               None,
                               "NO_MIN_MAJ",
                               "NO_PERCENT",
                               None)

#### Rename fields

In [ ]:
[f.name for f in arcpy.ListFields("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat_TotP")]

In [ ]:
fin_flds = [f.name for f in arcpy.ListFields("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat_TotP")]

In [ ]:
len_flds = [i for i in range(0,len(fin_flds))]
len(len_flds)

In [ ]:
new_fld_names = ["OBJECTID","Shape","Comp_num", "TARGET_FID", "all_GISCO",
                 "Ele_OBJECTID", "Ele_all_GISCO", "Ele_ZONE_CODE", "Ele_COUNT", "Ele_AREA", "Ele_MIN", "Ele_MAX", "Ele_MEAN",
                 "Sl_OBJECTID","Sl_all_GISCO", "Sl_ZONE_CODE", "Sl_COUNT", "Sl_AREA", "Sl_MIN", "Sl_MAX", "Sl_MEAN",
                 "Temp_OBJECTID", "Temp_all_GISCO", "Temp_ZONE_CODE", "Temp_COUNT", "Temp_AREA", "Temp_MIN", "Temp_MAX", "Temp_MEAN",
                 "Temp_MIN_C", "Temp_MAX_C", "Temp_MEAN_C",
                 "Area_km",
                 "Cyc_len_m", "Cyc_len_km", "Cyc_line_num",
                 "NEAR_FID", "NEAR_DIST_m", "NEAR_X", "NEAR_Y", "NEAR_ANGLE",
                 "timestamp", "name", "lat", "lon", "bikes", "spaces", "installed", "locked", "temporary", "total_dock", "Shape_Length", "Shape_Area",
                 "Pop_Pts_Tot", "Pop_Pts_Num" ]
len(new_fld_names)

In [ ]:
fin_flds_dicc = {k:v for k,v in zip(len_flds,new_fld_names)}
fin_flds_dicc

In [ ]:
ch_fin_flds_dicc = {}
for idx, name in enumerate(fin_flds):
    print(idx,name)
    ch_fin_flds_dicc[idx] = name

In [ ]:
for k1,v1 in ch_fin_flds_dicc.items():
    for k2,v2 in fin_flds_dicc.items():
        if k1 == k2:
            arcpy.management.AlterField("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat_TotP",
                                v1,
                                v2,
                                v2,
                                )

#### MBG - Company points (Join)

In [ ]:
arcpy.analysis.SpatialJoin("MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_Stat_TotP",
                           "Company_Points_EU_LCC",
                           r"E:\ESF MU\Bikesharing\ESF_data\ESF_data_view\MBG_units_characteristics.gdb\MBG_Pts_units_Ele_Sl_Tem_Ar_Cycl_TotP_Comp",
                           join_type="KEEP_ALL",
                           join_operation="JOIN_ONE_TO_MANY",
                           match_option="INTERSECT"
                           )